In [1]:
import pandas as pd
import numpy as np
from copy import copy

### データの読み込み

In [2]:
train = pd.read_csv('tutorial_preprocessed_train.csv', parse_dates=['date'])
weather = pd.read_csv('tutorial_preprocessed_weather.csv')
test = pd.read_csv('tutorial_preprocessed_test.csv', parse_dates=['date'])

train['month'] = train.date.dt.month
train['day'] = train.date.dt.day
train['early_and_late'] = 0
train['early_and_late'] = train['early_and_late'].mask(train['day'] > 15, 1)

test['month'] = test.date.dt.month
test['day'] = train.date.dt.day
test['early_and_late'] = 0
test['early_and_late'] = test['early_and_late'].mask(test['day'] > 15, 1)

display(train.head())
display(test.head())

,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late
0,だいこん,2005-11-07,201445.0,735.0,千葉,2005,45,11,7,0
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,2005,45,11,8,0
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,2005,45,11,10,0
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,2005,45,11,11,0
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,2005,45,11,12,0


,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late
0,だいこん,2022-11-01,NaN,NaN,千葉_全国_横浜,2022,44,11,7,0
1,だいこん,2022-11-02,NaN,NaN,千葉_全国_横浜,2022,44,11,8,0
2,だいこん,2022-11-04,NaN,NaN,千葉_全国_横浜,2022,44,11,10,0
3,だいこん,2022-11-05,NaN,NaN,千葉_全国_横浜,2022,44,11,11,0
4,だいこん,2022-11-07,NaN,NaN,千葉_全国_横浜,2022,45,11,12,0


### 1, 2, 3, 6, 9, 12か月前の気象情報を特徴量として追加

In [3]:
for i in [0, 1, 2, 3, 6, 9, 12]:
    cweather = copy(weather)
    cweather.month += 1
    cweather.loc[lambda x: x.month==13] = 1
    cweather.columns = [f'{c}_lag{i}' for c in cweather.columns]
    cweather = cweather.rename(columns={f'year_lag{i}': 'year', f'month_lag{i}': 'month', f'area_lag{i}': 'area'})
    train = train.merge(cweather, how='left', on=['year', 'month', 'area'])
    test = test.merge(cweather, how='left', on=['year', 'month', 'area'])
train.columns

Index(['kind', 'date', 'amount', 'mode_price', 'area', 'year', 'weekno',
       'month', 'day', 'early_and_late',
       ...
       'min_min_temp_lag12', 'mean_sum_rain_lag12', 'max_sum_rain_lag12',
       'min_sum_rain_lag12', 'mean_sun_time_lag12', 'max_sun_time_lag12',
       'min_sun_time_lag12', 'mean_mean_humid_lag12', 'max_mean_humid_lag12',
       'min_mean_humid_lag12'],
      dtype='object', length=136)

### 過去30日間、過去365日間の平均卸値を特徴量として追加

In [4]:
# mode_price_7days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_7days_ago'})
# mode_price_7days_ago.date += pd.Timedelta(days=7)
# mode_price_7days_ago['year'] = mode_price_7days_ago.date.dt.year
# mode_price_7days_ago['weekno'] = mode_price_7days_ago.date.dt.week
# mode_price_7days_ago = mode_price_7days_ago.groupby(['kind', 'year', 'weekno']).mean().reset_index()
# train = train.merge(mode_price_7days_ago, how='left', on=['kind', 'year', 'weekno'])
# test = test.merge(mode_price_7days_ago, how='left', on=['kind', 'year', 'weekno'])

mode_price_7days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_7days_ago'})
mode_price_7days_ago.date += pd.Timedelta(days=7)
mode_price_7days_ago['year'] = mode_price_7days_ago.date.dt.year
mode_price_7days_ago['month'] = mode_price_7days_ago.date.dt.month
mode_price_7days_ago = mode_price_7days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
train = train.merge(mode_price_7days_ago, how='left', on=['kind', 'year', 'month'])
test = test.merge(mode_price_7days_ago, how='left', on=['kind', 'year', 'month'])

mode_price_14days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_14days_ago'})
mode_price_14days_ago.date += pd.Timedelta(days=14)
mode_price_14days_ago['year'] = mode_price_14days_ago.date.dt.year
mode_price_14days_ago['month'] = mode_price_14days_ago.date.dt.month
mode_price_14days_ago = mode_price_14days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
train = train.merge(mode_price_14days_ago, how='left', on=['kind', 'year', 'month'])
test = test.merge(mode_price_14days_ago, how='left', on=['kind', 'year', 'month'])

mode_price_21days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_21days_ago'})
mode_price_21days_ago.date += pd.Timedelta(days=21)
mode_price_21days_ago['year'] = mode_price_21days_ago.date.dt.year
mode_price_21days_ago['month'] = mode_price_21days_ago.date.dt.month
mode_price_21days_ago = mode_price_21days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
train = train.merge(mode_price_21days_ago, how='left', on=['kind', 'year', 'month'])
test = test.merge(mode_price_21days_ago, how='left', on=['kind', 'year', 'month'])

mode_price_30days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_30days_ago'})
mode_price_30days_ago.date += pd.Timedelta(days=30)
mode_price_30days_ago['year'] = mode_price_30days_ago.date.dt.year
mode_price_30days_ago['month'] = mode_price_30days_ago.date.dt.month
mode_price_30days_ago = mode_price_30days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
train = train.merge(mode_price_30days_ago, how='left', on=['kind', 'year', 'month'])
test = test.merge(mode_price_30days_ago, how='left', on=['kind', 'year', 'month'])

mode_price_365days_ago = train[['kind', 'date', 'mode_price']].rename(columns={'mode_price': 'mode_price_365days_ago'})
mode_price_365days_ago.date += pd.Timedelta(days=365)
mode_price_365days_ago.date.max(), mode_price_365days_ago.columns
mode_price_365days_ago['year'] = mode_price_365days_ago.date.dt.year
mode_price_365days_ago['month'] = mode_price_365days_ago.date.dt.month
mode_price_365days_ago = mode_price_365days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
train = train.merge(mode_price_365days_ago, how='left', on=['kind', 'year', 'month'])
test = test.merge(mode_price_365days_ago, how='left', on=['kind', 'year', 'month'])

display(train.head())
display(test.head())

/var/folders/yz/_t9kmhr145v3b1nm0_gpsdw40000gn/T/ipykernel_18015/3329289091.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mode_price_7days_ago = mode_price_7days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
/var/folders/yz/_t9kmhr145v3b1nm0_gpsdw40000gn/T/ipykernel_18015/3329289091.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mode_price_30days_ago = mode_price_30days_ago.groupby(['kind', 'year', 'month']).mean().reset_index()
/var/folders/yz/_t9kmhr145v3b1nm0_gpsdw40000gn/T/ipykernel_18015/3329289091.py:30: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mea

,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,min_sum_rain_lag12,mean_sun_time_lag12,max_sun_time_lag12,min_sun_time_lag12,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago
0,だいこん,2005-11-07,201445.0,735.0,千葉,2005,45,11,7,0,...,0.0,3.577419,11.100000,0.0,79.064516,92.000000,60.000000,609.807692,NaN,NaN
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,2005,45,11,8,0,...,0.0,4.408501,10.761458,0.0,74.557796,90.895833,57.635417,609.807692,NaN,NaN
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,2005,45,11,10,0,...,0.0,4.408501,10.761458,0.0,74.557796,90.895833,57.635417,609.807692,NaN,NaN
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,2005,45,11,11,0,...,0.0,4.316129,10.900000,0.0,76.322581,92.000000,60.000000,609.807692,NaN,NaN
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,2005,45,11,12,0,...,0.0,4.316129,10.900000,0.0,76.322581,92.000000,60.000000,609.807692,NaN,NaN


,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,min_sum_rain_lag12,mean_sun_time_lag12,max_sun_time_lag12,min_sun_time_lag12,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago
0,だいこん,2022-11-01,NaN,NaN,千葉_全国_横浜,2022,44,11,7,0,...,0.0,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0
1,だいこん,2022-11-02,NaN,NaN,千葉_全国_横浜,2022,44,11,8,0,...,0.0,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0
2,だいこん,2022-11-04,NaN,NaN,千葉_全国_横浜,2022,44,11,10,0,...,0.0,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0
3,だいこん,2022-11-05,NaN,NaN,千葉_全国_横浜,2022,44,11,11,0,...,0.0,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0
4,だいこん,2022-11-07,NaN,NaN,千葉_全国_横浜,2022,45,11,12,0,...,0.0,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0


### 野菜毎にデータ数が同じになるように cross_validation

In [5]:
# 訓練データを2015年6月1日以降に限定
train = train[lambda x: x.date >= '2015-06-01']
#train = train[lambda x: x.date >= '2016-05-01']

from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits)

train['fold'] = -1
train_fold = []
for kind in test.kind.unique():
    train_kind = train[lambda x: x.kind==kind].copy()
    for i, (_, test_index) in enumerate(kf.split(train_kind)):
        train_kind.iloc[test_index, -1] = i
        train_fold.append(train_kind)
train_fold = pd.concat(train_fold, axis=0)

display(train_fold.groupby(['kind', 'fold']).count())
display(train_fold.groupby('fold').count())
display(train_fold.groupby(['kind', 'fold'])['date'].apply(lambda x: [x.min(), x.max()]))

date  amount  mode_price  area  year  weekno  month   day  \
kind fold                                                              
かぼちゃ 0     1635    1635        1635  1635  1635    1635   1635  1635   
     1     1635    1635        1635  1635  1635    1635   1635  1635   
     2     1635    1635        1635  1635  1635    1635   1635  1635   
     3     1635    1635        1635  1635  1635    1635   1635  1635   
     4     1630    1630        1630  1630  1630    1630   1630  1630   
...         ...     ...         ...   ...   ...     ...    ...   ...   
レタス  0     1920    1920        1920  1920  1920    1920   1920  1920   
     1     1920    1920        1920  1920  1920    1920   1920  1920   
     2     1915    1915        1915  1915  1915    1915   1915  1915   
     3     1915    1915        1915  1915  1915    1915   1915  1915   
     4     1915    1915        1915  1915  1915    1915   1915  1915   

           early_and_late  mean_mean_temp_lag0  ...  min_sum_rain_lag12  \
kind fold                                       ...                       
かぼちゃ 0               1635                 1635  ...                1635   
     1               1635                 1635  ...                1635   
     2               1635                 1635  ...                1635   
     3               1635                 1635  ...                1635   
     4               1630                 1630  ...                1630   
...                   ...                  ...  ...                 ...   
レタス  0               1920                 1820  ...                1820   
     1               1920                 1725  ...                1725   
     2               1915                 1820  ...                1820   
     3               1915                 1725  ...                1725   
     4               1915                 1815  ...                1815   

           mean_sun_time_lag12  max_sun_time_lag12  min_sun_time_lag12  \
kind fold                                                                
かぼちゃ 0                    1635                1635                1635   
     1                    1635                1635                1635   
     2                    1635                1635                1635   
     3                    1635                1635                1635   
     4                    1630                1630                1630   
...                        ...                 ...                 ...   
レタス  0                    1820                1820                1820   
     1                    1725                1725                1725   
     2                    1820                1820                1820   
     3                    1725                1725                1725   
     4                    1815                1815                1815   

           mean_mean_humid_lag12  max_mean_humid_lag12  min_mean_humid_lag12  \
kind fold                                                                      
かぼちゃ 0                      1635                  1635                  1635   
     1                      1635                  1635                  1635   
     2                      1635                  1635                  1635   
     3                      1635                  1635                  1635   
     4                      1630                  1630                  1630   
...                          ...                   ...                   ...   
レタス  0                      1820                  1820                  1820   
     1                      1725                  1725                  1725   
     2                      1820                  1820                  1820   
     3                      1725                  1725                  1725   
     4                      1815                  1815                  1815   

           mode_price_7days_ago  mode_price_30days_ago  mode_price_365days_ago  
kind fold    

,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,min_sum_rain_lag12,mean_sun_time_lag12,max_sun_time_lag12,min_sun_time_lag12,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago
fold,,,,,,,,,,,,,,,,,,,,,
0,27115,27115,27115,27115,27115,27115,27115,27115,27115,27115,...,25715,25715,25715,25715,25715,25715,25715,27115,26710,25995
1,27115,27115,27115,27115,27115,27115,27115,27115,27115,27115,...,24385,24385,24385,24385,24385,24385,24385,27115,26935,27115
2,27060,27060,27060,27060,27060,27060,27060,27060,27060,27060,...,25730,25730,25730,25730,25730,25730,25730,27060,26690,27060
3,27060,27060,27060,27060,27060,27060,27060,27060,27060,27060,...,24400,24400,24400,24400,24400,24400,24400,27060,26775,27060
4,27055,27055,27055,27055,27055,27055,27055,27055,27055,27055,...,25655,25655,25655,25655,25655,25655,25655,27055,26680,27055


kind  fold
かぼちゃ  0       [2015-06-01 00:00:00, 2016-10-15 00:00:00]
      1       [2016-10-17 00:00:00, 2018-05-08 00:00:00]
      2       [2018-05-10 00:00:00, 2019-10-08 00:00:00]
      3       [2019-10-10 00:00:00, 2021-05-21 00:00:00]
      4       [2021-05-22 00:00:00, 2022-10-31 00:00:00]
                                 ...                    
レタス   0       [2015-06-01 00:00:00, 2016-11-07 00:00:00]
      1       [2016-11-08 00:00:00, 2018-04-26 00:00:00]
      2       [2018-04-27 00:00:00, 2019-10-23 00:00:00]
      3       [2019-10-24 00:00:00, 2021-04-27 00:00:00]
      4       [2021-04-28 00:00:00, 2022-10-31 00:00:00]
Name: date, Length: 75, dtype: object

### kindをラベルエンコーディング

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_fold['kind_le'] = le.fit_transform(train_fold.kind)
test['kind_le'] = le.transform(test.kind)
display(train_fold.head())
display(test.head())

,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,max_sun_time_lag12,min_sun_time_lag12,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago,fold,kind_le
2585,だいこん,2015-06-01,175714.0,864.0,千葉_青森,2015,23,6,1,0,...,13.150000,0.000000,65.516129,86.500000,42.500000,1037.045455,1141.714286,826.071429,0,4
2586,だいこん,2015-06-02,167112.0,756.0,千葉_青森,2015,23,6,2,0,...,13.150000,0.000000,65.516129,86.500000,42.500000,1037.045455,1141.714286,826.071429,0,4
2587,だいこん,2015-06-04,202752.0,891.0,千葉_青森,2015,23,6,4,0,...,13.150000,0.000000,65.516129,86.500000,42.500000,1037.045455,1141.714286,826.071429,0,4
2588,だいこん,2015-06-05,142763.0,648.0,千葉_全国_青森,2015,23,6,5,0,...,13.155208,0.007292,64.958333,86.947917,42.270833,1037.045455,1141.714286,826.071429,0,4
2589,だいこん,2015-06-06,174205.0,810.0,千葉_青森,2015,23,6,6,0,...,13.150000,0.000000,65.516129,86.500000,42.500000,1037.045455,1141.714286,826.071429,0,4


,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,mean_sun_time_lag12,max_sun_time_lag12,min_sun_time_lag12,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago,kind_le
0,だいこん,2022-11-01,NaN,NaN,千葉_全国_横浜,2022,44,11,7,0,...,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0,4
1,だいこん,2022-11-02,NaN,NaN,千葉_全国_横浜,2022,44,11,8,0,...,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0,4
2,だいこん,2022-11-04,NaN,NaN,千葉_全国_横浜,2022,44,11,10,0,...,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0,4
3,だいこん,2022-11-05,NaN,NaN,千葉_全国_横浜,2022,44,11,11,0,...,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0,4
4,だいこん,2022-11-07,NaN,NaN,千葉_全国_横浜,2022,45,11,12,0,...,4.802148,10.76875,0.0,71.733724,91.34375,53.177083,658.8,984.857143,648.0,4


### 予測

In [7]:
import lightgbm as lgb

seed = 1234

params = {
    'boosting_type': 'gbdt',
    'metric': 'None',
    'objective': 'regression',
    'seed': seed,
    'learning_rate': 0.007,
}

import numpy as np

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [8]:
target = 'mode_price'
pred_train = np.zeros(len(train_fold))
pred_test = []
for fold in range(n_splits):
    print(f'---- {fold} ----')
    drop_cols = ['kind', 'area', 'date', 'amount']#, 'area_le']
    train_X = train_fold.drop(drop_cols, axis=1).loc[lambda x: x.fold!=fold].drop('fold', axis=1)
    train_y = train_X.pop(target)
    valid_X = train_fold.drop(drop_cols, axis=1).loc[lambda x: x.fold==fold].drop('fold', axis=1)
    valid_y = valid_X.pop(target)
    train_w = 1 / np.square(train_y)
    valid_w = 1 / np.square(valid_y)
    feature_name = train_X.columns.tolist()
    train = lgb.Dataset(
        data=train_X,
        label=train_y,
        feature_name=feature_name,
        weight = train_w,
    )
    valid = lgb.Dataset(
        data=valid_X,
        label=valid_y,
        feature_name=feature_name,
        weight = valid_w,
    )
    model = lgb.train(
        params,
        train,
        valid_sets = [valid],
        num_boost_round = 1000,
        verbose_eval = 100,
        early_stopping_rounds = 100,
        feval = feval_rmspe,
	)
    pred_train[train_fold.fold==fold] = model.predict(valid_X)
    pred_test.append(model.predict(test[feature_name]))

---- 0 ----


/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30029
[LightGBM] [Info] Number of data points in the train set: 108290, number of used features: 135
[LightGBM] [Info] Start training from score 109.760768
Training until validation scores don't improve for 100 rounds
[100]	valid_0's RMSPE: 0.451461
[200]	valid_0's RMSPE: 0.281207
[300]	valid_0's RMSPE: 0.216012
[400]	valid_0's RMSPE: 0.193849
[500]	valid_0's RMSPE: 0.186538
[600]	valid_0's RMSPE: 0.183804
[700]	valid_0's RMSPE: 0.182349
[800]	valid_0's RMSPE: 0.181955
[900]	valid_0's RMSPE: 0.181557
[1000]	valid_0's RMSPE: 0.181971
Did not meet early stopping. Best iteration is:
[917]	valid_0's RMSPE: 0.18147
---- 1 ----


/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.133755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30009
[LightGBM] [Info] Number of data points in the train set: 108290, number of used features: 135
[LightGBM] [Info] Start training from score 107.072764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's RMSPE: 0.435077
[200]	valid_0's RMSPE: 0.264841
[300]	valid_0's RMSPE: 0.201531
[400]	valid_0's RMSPE: 0.181649
[500]	valid_0's RMSPE: 0.176143
[600]	valid_0's RMSPE: 0.174213
[700]	valid_0's RMSPE: 0.174131
[800]	valid_0's RMSPE: 0.173665
[900]	valid_0's RMSPE: 0.173224
[1000]	valid_0's RMSPE: 0.17316
Did not meet early stopping. Best iteration is:
[988]	valid_0's RMSPE: 0.173142
---- 2 ----


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.188438 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29904
[LightGBM] [Info] Number of data points in the train set: 108345, number of used features: 135
[LightGBM] [Info] Start training from score 108.679755
Training until validation scores don't improve for 100 rounds
[100]	valid_0's RMSPE: 0.426347
[200]	valid_0's RMSPE: 0.257773
[300]	valid_0's RMSPE: 0.195333
[400]	valid_0's RMSPE: 0.17653
[500]	valid_0's RMSPE: 0.171576
[600]	valid_0's RMSPE: 0.170766
[700]	valid_0's RMSPE: 0.170729
Early stopping, best iteration is:
[684]	valid_0's RMSPE: 0.170607
---- 3 ----


/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055328 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29973
[LightGBM] [Info] Number of data points in the train set: 108345, number of used features: 135
[LightGBM] [Info] Start training from score 106.813536
Training until validation scores don't improve for 100 rounds
[100]	valid_0's RMSPE: 0.428304
[200]	valid_0's RMSPE: 0.26896
[300]	valid_0's RMSPE: 0.215137
[400]	valid_0's RMSPE: 0.199897
[500]	valid_0's RMSPE: 0.195793
[600]	valid_0's RMSPE: 0.194629
[700]	valid_0's RMSPE: 0.194162
[800]	valid_0's RMSPE: 0.193554
Early stopping, best iteration is:
[779]	valid_0's RMSPE: 0.193499
---- 4 ----


/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/takuma/Documents/vegetable/.venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29974
[LightGBM] [Info] Number of data points in the train set: 108350, number of used features: 135
[LightGBM] [Info] Start training from score 109.548711
Training until validation scores don't improve for 100 rounds
[100]	valid_0's RMSPE: 0.449387
[200]	valid_0's RMSPE: 0.293355
[300]	valid_0's RMSPE: 0.237671
[400]	valid_0's RMSPE: 0.219772
[500]	valid_0's RMSPE: 0.214292
[600]	valid_0's RMSPE: 0.212228
[700]	valid_0's RMSPE: 0.211597
[800]	valid_0's RMSPE: 0.211134
[900]	valid_0's RMSPE: 0.21094
[1000]	valid_0's RMSPE: 0.210752
Did not meet early stopping. Best iteration is:
[1000]	valid_0's RMSPE: 0.210752


### CV確認

In [9]:
train_fold['pred'] = pred_train
print(rmspe(train_fold.mode_price, train_fold.pred))

for k in train_fold.kind.unique():
    print(k, len(test[lambda x: x.kind==k]), rmspe(train_fold.loc[lambda x: x.kind==k, 'mode_price'], train_fold.loc[lambda x: x.kind==k, 'pred']))
    
for m in train_fold.month.unique():
    print(m, len(test[lambda x: x.month==m]), rmspe(train_fold.loc[lambda x: x.month==m, 'mode_price'], train_fold.loc[lambda x: x.month==m, 'pred'])) 

0.18647219090371775
だいこん 21 0.18364754862878052
にんじん 21 0.19522629692330692
キャベツ 21 0.17540807477255219
レタス 21 0.2392674153167867
はくさい 21 0.25109754206111595
ほうれんそう 21 0.1637270890013704
ねぎ 21 0.17106637265081295
きゅうり 21 0.17571908369384204
かぼちゃ 21 0.24535998360863967
トマト 21 0.1531783519958535
ピーマン 21 0.16234748264446033
さといも 21 0.18243149594804292
たまねぎ 21 0.2023956066229816
なましいたけ 21 0.08209879405761138
ミニトマト 21 0.12688956338777013
6 0 0.18727404846527024
7 0 0.18966610047895852
8 0 0.21697856525251485
9 0 0.1764799229082324
10 0 0.183804380751674
11 315 0.206918918171135
12 0 0.18350758324709593
1 0 0.16452793365627041
2 0 0.1504024633882814
3 0 0.17464612683619946
4 0 0.194805812648746
5 0 0.19582926831396272


In [10]:
train_fold['resid'] = (train_fold['mode_price'] - train_fold['pred']).abs() / train_fold['mode_price'] 
train_fold.sort_values('resid', ascending=False)

,kind,date,amount,mode_price,area,year,weekno,month,day,early_and_late,...,mean_mean_humid_lag12,max_mean_humid_lag12,min_mean_humid_lag12,mode_price_7days_ago,mode_price_30days_ago,mode_price_365days_ago,fold,kind_le,pred,resid
34011,ねぎ,2019-07-27,74068.0,540.0,千葉_全国_水戸,2019,30,7,27,1,...,76.409028,95.114583,49.59375,1872.818182,1968.545455,1620.000000,2,7,1592.009117,1.948165
34011,ねぎ,2019-07-27,74068.0,540.0,千葉_全国_水戸,2019,30,7,27,1,...,76.409028,95.114583,49.59375,1872.818182,1968.545455,1620.000000,2,7,1592.009117,1.948165
34011,ねぎ,2019-07-27,74068.0,540.0,千葉_全国_水戸,2019,30,7,27,1,...,76.409028,95.114583,49.59375,1872.818182,1968.545455,1620.000000,2,7,1592.009117,1.948165
34011,ねぎ,2019-07-27,74068.0,540.0,千葉_全国_水戸,2019,30,7,27,1,...,76.409028,95.114583,49.59375,1872.818182,1968.545455,1620.000000,2,7,1592.009117,1.948165
34011,ねぎ,2019-07-27,74068.0,540.0,千葉_全国_水戸,2019,30,7,27,1,...,76.409028,95.114583,49.59375,1872.818182,1968.545455,1620.000000,2,7,1592.009117,1.948165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62397,なましいたけ,2021-10-14,20782.0,108.0,全国_秋田,2021,41,10,14,0,...,74.787500,94.000000,60.53125,112.431818,113.619048,107.521739,4,5,107.999283,0.000007
62397,なましいたけ,2021-10-14,20782.0,108.0,全国_秋田,2021,41,10,14,0,...,74.787500,94.000000,60.53125,112.431818,113.619048,107.521739,4,5,107.999283,0.000007
62397,なましいたけ,2021-10-14,20782.0,108.0,全国_秋田,2021,41,10,14,0,...,74.787500,94.000000,60.53125,112.431818,113.619048,107.521739,4,5,107.999283,0.000007
62397,なましいたけ,2021-10-14,20782.0,108.0,全国_秋田,2021,41,10,14,0,...,74.787500,94.000000,60.53125,112.431818,113.619048,107.521739,4,5,107.999283,0.000007
